# Modelagem de tópicos – *Latent Dirichlet Allocation (LDA)*
## Processamento de Linguagem Natural
Nesta aula trabalharemos com uma tarefa de PLN muito popular, a Modelagem de Tópicos. O objetivo é que ao final desta aula você:
1. Entenda o que é a Modelagem de tópicos
2. O que a diferencia de outras técnicas?
3. Compreenda e aplique o algoritmo *Latent Dirichlet Allocation (**LDA**)*

### **O que é Modelagem de tópicos?**

É um modelo estatístico para descobrir "tópicos" que ocorrem em uma coleção de documentos.

Uma ferramenta de modelagem de tópicos tenta "injetar" significado semântico ao **procurar padrões de uso de palavras em meio ao texto**. Para o computador, um tópico é uma lista de palavras que ocorrem em jeitos estatísticamente significativos.

Estes algoritmos não entendem o significado das palavras, eles apenas assumem que qualquer pedaço de texto é composto ao selecionarmos palavras de cestos de palavras, onde **cada cesto corresponde a um tópico**. Neste caso então, é possível decompor o texto em prováveis cestos de onde as palarvas vieram.


![LDA - Topic Modeling - Probabilistic
Topic Models - Blei, 2012](https://docs.google.com/uc?export=download&id=1wpyDoKwzJ8ynMmB_eM3ryG4mfjeNy0-X)

#### O TF-IDF também consegue identificar *keywords*, então qual a diferença entre as duas técnicas?

Apesar das duas técnicas conseguirem isolar e rankear termos importantes em um documento, as duas são muito diferentes entre si.

O **TF-IDF** por exemplo, é mais apropriado quando você quer uma visão ampla do seu corpus ainda na fase de **análise exploratória dos dados**, pois o algoritmo é transparente em suas pontuações e pode ser facilmente replicável.

É importante lembrar, que **os tópicos gerados na modelagem de tópicos nem sempre serão coerentes**. Mesmo assim, a modelagem de tópicos também pode ser útil para explorar os dados, pois conseguem sugerir categorias amplas ou clusters de textos dentro da coleção.

Os modelos de tópicos são especialmente atraentes porque os documentos recebem pontuações de como eles se encaixam em cada tópico e porque os tópicos são representados como listas de termos co-ocorrentes, o que **fornece um forte senso de como os termos se relacionam com os agrupamentos**. No entanto, o modelo probabilístico por trás dos modelos de tópicos é sofisticado e é fácil distorcer seus resultados se você não entender o que está fazendo. Já a matemática por trás do tf-idf é simples o suficiente para ser representada em uma planilha.





#### Mas quando usar Modelagem de tópicos então?

Antes de utilizar, você deve entender a real utilidade desta técnica. Por exemplo, se você terá acesso a uma pequena coleção de documentos, ou até mesmo um único documento, basta utilizar algo que conte a frequência das palavras (BoW, TF-IDF) que será o suficiente.

Já se você tem acesso a **centenas de documentos**, e deseja **compreender** melhor do que se tratam tais dados sem ter de realizar a leitura destes documentos, então a **Modelagem de tópicos provavelmente seja uma boa abordagem**.

##### Exemplo

Um exemplo seria a análise de discursos de um político, onde o algoritmo poderia retornar uma lista de tópicos e palavras-chave que compõe estes tópicos.

- *emprego empregos perda desemprego*
- *economia mercado banco moeda*
- *covid pandemia gripe quarentena*
- *pt lula dilma esquerda*

Ao verificar estas palavras-chave conseguimos verificar que o político em questão está preocupado com empregos, economia, a pandemia e com a esquerda.

### **Latent Dirichlet Allocation (LDA)**

Latent Dirichlet Allocation (LDA) é o método mais popular para
modelagem de tópicos. Baseia-se na distribuição probabilística do matemático Dirichlet. O LDA segue as seguintes intuiçoes:
- Documentos com tópicos similares usam grupos similares de palavras
- Tópicos podem então ser achados ao buscarmos grupos de palavras que acontecem juntas frequentemente em documentos do corpus

#### Dados
Nós usaremos os dados de notícias disponíveis na própria biblioteca sklearn. As notícias são em inglês, mas o código funciona exatamante da mesma maneira caso usemos em pt-br.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
# Imprime o tamanho do dataset
len(documents)

11314

In [ ]:
# Imprime os grupos de notícias
dataset.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

#### Pré-processamento

In [ ]:
import pandas as pd

noticiasDf = pd.DataFrame({'documento':documents})

# Remove tudo que não seja letra
noticiasDf['documento_limpo'] = noticiasDf['documento'].str.replace("[^a-zA-Z#]", " ")

# Remove palavras curtas
noticiasDf['documento_limpo'] = noticiasDf['documento_limpo'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# Transforma tudo em minúsculas
noticiasDf['documento_limpo'] = noticiasDf['documento_limpo'].apply(lambda x: x.lower())



> **PERGUNTA**: Por que você acha que estas ações podem ser úteis?
Ajuda na redução da dimensionalidade, diminiu o tamanho doi vocabulário, pode ser ruim por retirar informações relevantes, como a marca HP em texto do domínio de TI.



In [ ]:
noticiasDf
# Saida de texto comum e texto limpo

,documento,documento_limpo
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased. what disagr...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...","yeah, expect people read faq, etc. actually ac..."
2,Although I realize that principle is not one o...,although realize that principle your strongest...
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...
4,"Well, I will have to change the scoring on my ...","well, will have change scoring playoff pool. u..."
...,...,...
11309,"Danny Rubenstein, an Israeli journalist, will ...","danny rubenstein, israeli journalist, will spe..."
11310,\n,
11311,\nI agree. Home runs off Clemens are always m...,agree. home runs clemens always memorable. kin...
11312,I used HP DeskJet with Orange Micros Grappler ...,used deskjet with orange micros grappler syste...


Vamos agora utilizar o **CountVectorizer** para construir nossa **matriz termo-documento**. Já aproveitaremos alguns recursos da biblioteca para realizar mais etapas de pré-processamento.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

mtd = cv.fit_transform(noticiasDf['documento_limpo'])

In [ ]:
# docs x words
mtd
# Tamanho da matriz com nro de documentos e palavras diferentes

<11314x38414 sparse matrix of type '<class 'numpy.int64'>'
	with 646919 stored elements in Compressed Sparse Row format>



> **PERGUNTA**: Para que servem os parâmetros passados para a função CountVectorizer? Servem para pré-processar e reduzir o tamanho da matriz.

Verifique a documentação e re-execute o código sem passar os parâmetros.



#### **LDA**
O LDA já é implementado pelo sklearn

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
# Define quantidade de tópicos/clusters, escolheu-se 20 pra verificar se algoritmo consegue encontrar mesmas categorias pré-estabelecidas no dataset
# Definir o random_state faz com que a aleatoriedade na seleção das palavras seja a mesma, toda vez que este código for executado
LDA = LatentDirichletAllocation(n_components=20,random_state=42) # quantos topicos a encontrar = 20 temáticas diferentes
# Pode demorar se há muitos dados
LDA.fit(mtd)

LatentDirichletAllocation(n_components=20, random_state=42)

Como obter o **vocabulário**?

In [ ]:
cv.get_feature_names_out()

array(['00', '000', '0000', ..., 'zyra', 'zyxel', 'zz'], dtype=object)

In [ ]:
len(cv.get_feature_names_out())

38414

In [ ]:
# Você pode obter qualquer palavra do vocabulário
cv.get_feature_names_out()[20078] #acessa pela posição do indice

'kilogram'

Como obter os **tópicos**?

In [ ]:
# Quantidade de tópicos
len(LDA.components_)

20

In [ ]:
# Cada tópico é um array com as probabilidades de cada palavra
LDA.components_

array([[2.00576726e+00, 1.15905250e+02, 5.00000004e-02, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000000e-02],
       [3.30116002e-01, 5.00000011e-02, 5.00000000e-02, ...,
        5.00000000e-02, 5.00000005e-02, 5.00000000e-02],
       [1.48514808e+00, 3.45391676e+01, 1.08061761e+00, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000000e-02],
       ...,
       [5.85356289e+02, 1.91499153e+02, 5.00000003e-02, ...,
        5.00000000e-02, 5.00000010e-02, 5.00000000e-02],
       [1.46676399e+01, 6.20821833e+01, 5.00000001e-02, ...,
        5.00000000e-02, 5.00000005e-02, 5.00000000e-02],
       [9.21494893e+00, 2.68553028e+01, 3.94692315e+00, ...,
        5.00000000e-02, 5.00000135e-02, 5.00000000e-02]])

In [ ]:
# tópicos x palavras
LDA.components_.shape

(20, 38414)

Como obter as **palavras com maior probabilidade** para cada tópico?

In [ ]:
# Obtém primeiro tópico - ainda não sabemos do que se trata
primeiro_topico = LDA.components_[0]

In [ ]:
# Obtemos os índices ordenados do menor pro maior
primeiro_topico.argsort()

array([19206, 20830, 20820, ..., 33778, 13379, 31983])



> **ENTENDA MELHOR**: Veja no exemplo a seguir o que o comando argsort() faz



In [ ]:
import numpy as np
arr = np.array([5, 50, 1, 10, 15])
# Ordena a posição dos elementos do array
arr.argsort()

array([2, 0, 3, 4, 1])

Voltando para nossos textos...
Agora sabemos a localização palavras com maior valor de probabilidade!

In [ ]:
# Obtém os últimos dez valores - que como ordenamos, serão os que tem maior valor de probabilidade
top_10 = primeiro_topico.argsort()[-10:]

In [ ]:
top_10

array([23892, 13741, 10964, 16246, 20579,  8678,  9030, 33778, 13379,
       31983])

Agora que temos os índices, basta imprimir os valores

In [ ]:
for i in top_10:
  print(cv.get_feature_names_out()[i])

national
escrow
data
government
launch
chip
clipper
technology
encryption
space


> **PERGUNTAS**: As palavras fazem sentido juntas? A qual tipo de tópico você acha que os documentos deste cluster pertencem?

As vezes os tópicos podem ter uma grande intersecção, ou pouco representativos. Não há um número mágico, você deve avaliar!




In [ ]:
# Imprime todos as top palavras de cada tópico
for i, topic in enumerate(LDA.components_):
  print("\n\n=== TOP 15 palavras - Tópico ", i)
  print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])



=== TOP 15 palavras - Tópico  0
['security', 'keys', 'enforcement', 'information', 'satellite', 'national', 'escrow', 'data', 'government', 'launch', 'chip', 'clipper', 'technology', 'encryption', 'space']


=== TOP 15 palavras - Tópico  1
['lib', 'work', 'value', 'engine', 'time', 'power', 'application', 'just', 've', 'problem', 'using', 'like', 'used', 'widget', 'window']


=== TOP 15 palavras - Tópico  2
['years', 'came', 'told', 'going', 've', 'went', 'think', 'didn', 'time', 'just', 'like', 'know', 'don', 'said', 'people']


=== TOP 15 palavras - Tópico  3
['muslim', 'russian', 'history', 'world', 'armenia', 'genocide', 'turks', 'greek', 'turkey', 'government', 'jews', 'people', 'armenians', 'turkish', 'armenian']


=== TOP 15 palavras - Tópico  4
['work', 'bit', 'need', 'thanks', 'don', 'problem', 'hard', 'does', 'just', 'know', 'disk', 'like', 'scsi', 'card', 'drive']


=== TOP 15 palavras - Tópico  5
['18', 'play', 'power', '13', '17', '14', '16', '20', '12', '11', '50', '15'

Como definir a **probabilidade de um DOCUMENTO pertencer a um tópico**?

Ou seja, como atribuir os tópicos descobertos para os documentos do corpus?

In [ ]:
# Primeiro, lembre-se que temos nossa matriz termo-documento
mtd

<11314x38414 sparse matrix of type '<class 'numpy.int64'>'
	with 646919 stored elements in Compressed Sparse Row format>

In [ ]:
# Também temos o DataFrame com os textos originais
noticiasDf

,documento,documento_limpo
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased. what disagr...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...","yeah, expect people read faq, etc. actually ac..."
2,Although I realize that principle is not one o...,although realize that principle your strongest...
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...
4,"Well, I will have to change the scoring on my ...","well, will have change scoring playoff pool. u..."
...,...,...
11309,"Danny Rubenstein, an Israeli journalist, will ...","danny rubenstein, israeli journalist, will spe..."
11310,\n,
11311,\nI agree. Home runs off Clemens are always m...,agree. home runs clemens always memorable. kin...
11312,I used HP DeskJet with Orange Micros Grappler ...,used deskjet with orange micros grappler syste...


Vamos criar uma nova coluna neste DataFrame com o número do tópico correspondente

In [ ]:
# Obtém as probabilidades de cada tópico por texto
topicos_prob = LDA.transform(mtd)

In [ ]:
# Textos x Tópicos
topicos_prob.shape

(11314, 20)

In [ ]:
# Probabilidades de cada tópico para o PRIMEIRO DOCUMENTO do corpus
topicos_prob[0]

array([0.00086207, 0.00086207, 0.07407759, 0.3981824 , 0.00086207,
       0.00086207, 0.00086207, 0.00086207, 0.00086207, 0.00086207,
       0.00086207, 0.16871744, 0.00086207, 0.00086207, 0.34522946,
       0.00086207, 0.00086207, 0.00086207, 0.00086207, 0.00086207])

Abra o documento em questão e verifique se o texto tem realmente relação com as palavras associadas ao tópico. Faça isso para vários documentos.

In [ ]:
noticiasDf['documento'][0]

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [ ]:
# Obtém a posição da maior probabilidade para cada documento e coloca na nova coluna
noticiasDf['topico'] = topicos_prob.argmax(axis=1)
noticiasDf

,documento,documento_limpo,topico
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased. what disagr...,3
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...","yeah, expect people read faq, etc. actually ac...",11
2,Although I realize that principle is not one o...,although realize that principle your strongest...,14
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...,10
4,"Well, I will have to change the scoring on my ...","well, will have change scoring playoff pool. u...",17
...,...,...,...
11309,"Danny Rubenstein, an Israeli journalist, will ...","danny rubenstein, israeli journalist, will spe...",17
11310,\n,,0
11311,\nI agree. Home runs off Clemens are always m...,agree. home runs clemens always memorable. kin...,9
11312,I used HP DeskJet with Orange Micros Grappler ...,used deskjet with orange micros grappler syste...,6


### ATIVIDADE PRÁTICA
Agora é com você, faça algumas alterações nas chamadas para analisar os resultados.

*   Tente reduzir o número de componentes e encontrar uma quantidade que faça mais sentido para você


In [ ]:
# Define quantidade de tópicos/clusters, escolheu-se 20 pra verificar se algoritmo consegue encontrar mesmas categorias pré-estabelecidas no dataset
# Definir o random_state faz com que a aleatoriedade na seleção das palavras seja a mesma, toda vez que este código for executado
LDA = LatentDirichletAllocation(n_components=12,random_state=42) # quantos topicos a encontrar = 20 temáticas diferentes
# Pode demorar se há muitos dados
LDA.fit(mtd)

LatentDirichletAllocation(n_components=12, random_state=42)

In [ ]:
# Quantidade de tópicos
len(LDA.components_)

12

In [ ]:
# Cada tópico é um array com as probabilidades de cada palavra
LDA.components_

array([[4.29081424e+01, 1.44976155e+02, 8.33350347e-02, ...,
        8.33333333e-02, 8.33333333e-02, 8.33333333e-02],
       [8.33364964e-02, 1.72843760e+01, 8.33340228e-02, ...,
        8.33333333e-02, 8.33333333e-02, 8.33333333e-02],
       [1.79134740e+00, 5.97662688e+01, 1.09996164e+00, ...,
        8.33333333e-02, 8.33364399e-02, 8.33333333e-02],
       ...,
       [5.44743691e+00, 3.85388257e+00, 8.33333333e-02, ...,
        8.33333333e-02, 8.33333333e-02, 8.33333333e-02],
       [4.79913228e+00, 2.49995856e+02, 8.33348574e-02, ...,
        8.33333333e-02, 8.33333333e-02, 8.33333333e-02],
       [2.86556807e+00, 1.34298054e+01, 8.33346845e-02, ...,
        8.33333333e-02, 2.08331769e+00, 8.33333333e-02]])

In [ ]:
# tópicos x palavras
LDA.components_.shape

(12, 38414)

In [ ]:
# Obtém primeiro tópico - ainda não sabemos do que se trata
primeiro_topico = LDA.components_[0]

In [ ]:
# Obtemos os índices ordenados do menor pro maior
primeiro_topico.argsort()

array([30388, 37540, 37544, ..., 31983, 33778, 13379])

In [ ]:
# Obtém os últimos dez valores - que como ordenamos, serão os que tem maior valor de probabilidade
top_10 = primeiro_topico.argsort()[-10:]

In [ ]:
top_10

array([ 9030, 16246, 23892, 10964, 30224, 18533, 27282, 31983, 33778,
       13379])

In [ ]:
# Imprime os valores do top 10
for i in top_10:
  print(cv.get_feature_names_out()[i])

clipper
government
national
data
sale
information
privacy
space
technology
encryption


In [ ]:
# Imprime todos as top palavras de cada tópico
for i, topic in enumerate(LDA.components_):
  print("\n\n=== TOP 15 palavras - Tópico ", i)
  print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])



=== TOP 15 palavras - Tópico  0
['administration', 'computer', 'university', 'research', 'security', 'clipper', 'government', 'national', 'data', 'sale', 'information', 'privacy', 'space', 'technology', 'encryption']


=== TOP 15 palavras - Tópico  1
['application', 'orbit', 'problem', 'high', 'just', 'earth', 'engine', 'using', 'used', 'power', 'like', 'window', 'time', 'nasa', 'space']


=== TOP 15 palavras - Tópico  2
['told', 'came', 'want', 'years', 've', 'went', 'didn', 'think', 'time', 'just', 'know', 'like', 'don', 'said', 'people']


=== TOP 15 palavras - Tópico  3
['genocide', 'mv', 'chz', 'ck', 'uw', 't7', 'c_', 'turks', 'turkey', 'armenians', 'w7', '55', 'cx', 'armenian', 'turkish']


=== TOP 15 palavras - Tópico  4
['need', 'thanks', 'work', 'chip', 'disk', 'problem', 'windows', 'scsi', 'don', 'does', 'just', 'know', 'card', 'like', 'drive']


=== TOP 15 palavras - Tópico  5
['50', '12', '30', '11', 'season', '15', '20', 'period', 'games', 'play', 'hockey', '10', 'game',

In [ ]:
# Obtém as probabilidades de cada tópico por texto
topicos_prob = LDA.transform(mtd)

In [ ]:
# Textos x Tópicos
topicos_prob.shape

(11314, 12)

In [ ]:
# Probabilidades de cada tópico para o PRIMEIRO DOCUMENTO do corpus
topicos_prob[0]

array([0.0014368 , 0.0014368 , 0.3304163 , 0.0014368 , 0.0014368 ,
       0.00143679, 0.00143679, 0.0014368 , 0.00143678, 0.00143683,
       0.47078543, 0.18586706])

In [ ]:
# Abre o documento
noticiasDf['documento'][0]

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [ ]:
# Probabilidades de cada tópico para o PRIMEIRO DOCUMENTO do corpus
topicos_prob[8]

array([0.00208341, 0.00208342, 0.65469234, 0.00208334, 0.00208343,
       0.00208335, 0.27776925, 0.00208345, 0.00208333, 0.0020834 ,
       0.04878784, 0.00208344])

In [ ]:
# Abre o documento
noticiasDf['documento'][8]

" Nobody is saying that you shouldn't be allowed to use msg.  Just\ndon't force it on others. If you have food that you want to \nenhance with msg just put the MSG on the table like salt.  It is\nthen the option of the eater to use it.  If you make a commerical\nproduct, just leave it out. You can include a packet (like some\nsalt packets) if you desire.\n\nSalt, pepper, mustard, ketchup, pickles ..... are table options.\nTreat MSG the same way.  I wouldn't shove my condiments down your\nthroat, don't shove yours down mine.\n\nWFL\n"

In [ ]:
# Obtém a posição da maior probabilidade para cada documento e coloca na nova coluna
noticiasDf['topico'] = topicos_prob.argmax(axis=1)
noticiasDf

,documento,documento_limpo,topico
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased. what disagr...,10
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...","yeah, expect people read faq, etc. actually ac...",11
2,Although I realize that principle is not one o...,although realize that principle your strongest...,10
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...,0
4,"Well, I will have to change the scoring on my ...","well, will have change scoring playoff pool. u...",9
...,...,...,...
11309,"Danny Rubenstein, an Israeli journalist, will ...","danny rubenstein, israeli journalist, will spe...",5
11310,\n,,0
11311,\nI agree. Home runs off Clemens are always m...,agree. home runs clemens always memorable. kin...,9
11312,I used HP DeskJet with Orange Micros Grappler ...,used deskjet with orange micros grappler syste...,6


## Referências e Material complementar

* [LDA - Artigo original](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)  
* [Modelagem de tópicos - Prof. Walmes Zeviani](http://www.leg.ufpr.br/~walmes/ensino/mintex/slides/08-topicos.pdf)
* [Tutorial de LSA - Latent Semantic Analysis](https://pessoalex.wordpress.com/2019/04/01/uma-introducao-a-modelagem-de-topicos-utilizando-analise-semantica-latente-em-python/)
* [LDA in Python – How to grid search best topic models?](https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/)
* [Topic modeling visualization – How to present the results of LDA models?](https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/)

Este notebook foi produzido por Prof. [Lucas Oliveira](http://lattes.cnpq.br/3611246009892500).